# Graph Data Science for Supply Chain
## Creating Informative Metrics and Analyzing Supply Chain Performance
![Neo4j version](https://img.shields.io/badge/Neo4j->=4.4.9-brightgreen)
![GDS version](https://img.shields.io/badge/GDS->=2.2-brightgreen)

This notebook compliments [part II of the Graph Data Science for Supply Chains Blog Series](https://neo4j.com/developer-blog/gds-supply-chains-metrics-performance-python/) . In this notebook we explore the application of graph data science to creating informative supply chain metrics and analyzing the performance of a logistics network. Specifically we will

1. Provide meaningful interpretations for Centrality metrics and Louvain communities within the context of supply chain and their relationship to performance and risks.
2. Show how to calculate Centrality metrics and Louvain communities using Neo4j Graph Data Science.
3. Demonstrate how these metrics can be used for downstream inference to understand if/how they relate to delays at different airports, providing insights into how the logistics network structure may affect freight delivery performance.


For a sample dataset we will use the “Cargo 2000” transport and logistics case study [[1]](#1). Cargo 2000 (re-branded as Cargo iQ in 2016) is an initiative of the International Air Transport Association (IATA) that aims to deliver a new quality management system for the air cargo industry.logistics-diagram.png

The below figure shows a model of the business processes covered in the IATA case study. It represents the business processes of a freight forwarding company, in which up to three smaller shipments from suppliers are consolidated and then shipped together to customers. The business process is structured into incoming and outgoing transport legs, with the overall objective that freight is delivered to customers in a timely manner.  You can find out more about the business model in the [first blog of this series](https://neo4j.com/developer-blog/supply-chain-neo4j-gds-bloom/) where we explored the dataset in Neo4j Bloom or from the [original data source]( https://s-cube-network.eu/c2k/).

<img src="img/logistics-diagram.png" alt="summary" width="1000"/>


## Prerequisites
- The Cargo 2000 case study dataset must be loaded into Neo4j. You can do so by running [this notebook](https://github.com/neo4j-product-examples/ds-supply-chain-use-cases/blob/main/modeling-eda-visualization-c2k/transform-and-load.ipynb). It should take no more than a few minutes to complete.

## References
<a id="1">[1]</a> A. Metzger, P. Leitner, D. Ivanovic, E. Schmieders, R. Franklin, M. Carro, S. Dustdar, and K. Pohl, “ Comparing and combining predictive business process monitoring techniques,” IEEE Trans. on Systems Man Cybernetics: Systems, 2015.


In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

## Connect to Neo4j Graph Data Science

In [4]:
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
gds = GraphDataScience('neo4j://localhost', auth=('neo4j', 'examplePassword'))

## Collapse Graph Data Model with `SENDS_TO` Relationships
Relationships going directly to/from airport nodes will allow for more direct calculation of centrality and community metrics according to transport routes.

In [5]:
gds.run_cypher('''
    MATCH(a1:Airport)<-[:LOCATED_AT]-(d1:DeparturePoint)-[r:TRANSPORT]->(d2:ArrivalWarehouse)-[:LOCATED_AT]->(a2:Airport)
    WITH a1, a2, count(r) AS flightCount
    MERGE (a1)-[s:SENDS_TO]->(a2)
    SET s.flightCount = flightCount
    RETURN count(s)
''')

,count(s)
0,1205


## Identifying and Interpreting Graph Based Metrics for Supply Chains

In [part I](https://neo4j.com/developer-blog/supply-chain-neo4j-gds-bloom/), we learned about graph algorithms relevant to supply chain analytics.  Below is a summary for how to interpret the results of these algorithms.  I will also add an additional centrality algorithm that we didn’t cover in part I called Eigenvector centrality.


- [Degree Centrality](https://neo4j.com/docs/graph-data-science/current/algorithms/degree-centrality) measures the __*operational load*__ for stages in your supply chain.  Stages with high operational load have to manage larger inflows and outflows and may be forced to reconcile conflicting schedules and priorities more often[[2]](#2). All else held constant, stages with higher operational load tend to __*require more resources*__ to run effectively.

- [Betweenness Centrality](https://neo4j.com/docs/graph-data-science/current/algorithms/betweenness-centrality/) measures the __*flow control*__ for stages in distribution and logistics networks. Stages with high Betweenness Centrality have more control over the flow of material and/or product because they connect many other stages together that may otherwise be disconnected or connected through much longer less efficient paths[[2]](#2). All else held constant, stages with higher flow control present __*higher risk for causing bottlenecks*__ in supply chains if they encounter delays or other issues.

- [Eigenvector Centrality](https://neo4j.com/docs/graph-data-science/current/algorithms/eigenvector-centrality/) measures the __*transitive influence*__ of stages in a supply chain[[2]](#2).  Stages with high Eigenvector Centrality are depended upon by other critical stages. These stages have high transitive influence because delays or other performance issues for these stages have a __*higher risk of carrying over or propagating to other critical stages*__ in the supply chain.

- [Louvain Community Detection](https://neo4j.com/docs/graph-data-science/current/algorithms/louvain/) finds __*regional interdependence*__  within the supply chain network by identifying groups of stages which have highly interconnected flows between them.  All else held constant, Stages within the same group have a stronger interdependence on each other relative to stages outside the group.

<a id="1">[2]</a> Christian Wallmann, Markus Gerschberger, The association between network centrality measures and supply chain performance: The case of distribution networks, Procedia Computer Science, Volume 180, 2021, Pages 172-179, ISSN 1877-0509, https://doi.org/10.1016/j.procs.2021.01.153.

## Calculating Centrality Metrics Using Neo4j Graph Data Science

In [6]:
# Create the in-memory graph projection
g, _ = gds.graph.project('proj', 'Airport', {'SENDS_TO':{'properties':['flightCount']}})

Loading:   0%|          | 0/100 [00:00<?, ?%/s]

In [7]:
# calculate and write out-degree centrality
gds.degree.write(g,relationshipWeightProperty='flightCount', writeProperty='outDegreeCentrality')
# calculate and write betweenness centrality
gds.betweenness.write(g, writeProperty='betweennessCentrality')
#calculate and write eigenvector centrality
gds.eigenvector.write(g,relationshipWeightProperty='flightCount', writeProperty='eigenvectorCentrality')
# drop the projected in-memory graph
g.drop()

graphName                                                         proj
database                                                         neo4j
memoryUsage                                                           
sizeInBytes                                                         -1
nodeCount                                                          237
relationshipCount                                                 1205
configuration        {'relationshipProjection': {'SENDS_TO': {'orie...
density                                                       0.021544
creationTime                       2023-01-21T17:11:14.229642000-05:00
modificationTime                   2023-01-21T17:11:14.513466000-05:00
schema               {'graphProperties': {}, 'relationships': {'SEN...
Name: 0, dtype: object

## Calculate In-Degree Centrality on REVERSED Orientation

In [8]:
g, _ = gds.graph.project('proj', 'Airport', {'SENDS_TO':{'orientation':'REVERSE', 'properties':['flightCount']}})
gds.degree.write(g,relationshipWeightProperty='flightCount', writeProperty='inDegreeCentrality')
g.drop()

graphName                                                         proj
database                                                         neo4j
memoryUsage                                                           
sizeInBytes                                                         -1
nodeCount                                                          237
relationshipCount                                                 1205
configuration        {'relationshipProjection': {'SENDS_TO': {'orie...
density                                                       0.021544
creationTime                       2023-01-21T17:11:18.323908000-05:00
modificationTime                   2023-01-21T17:11:18.395607000-05:00
schema               {'graphProperties': {}, 'relationships': {'SEN...
Name: 0, dtype: object

## Top 5 Airports for Each Centrality Metric

In [9]:
metrics = ['betweennessCentrality', 'outDegreeCentrality', 'inDegreeCentrality', 'eigenvectorCentrality']
top_n = 5
for metric in metrics:
    print('\n=======================================')
    print(f'Top {top_n} Airports for {metric}')
    print(gds.run_cypher(f'''
        MATCH(a:Airport)
        RETURN a.airportId AS airportId, a.name AS name, a.{metric} AS {metric}
        ORDER BY {metric} DESC LIMIT {top_n}
    '''))


Top 5 Airports for betweennessCentrality
   airportId         name  betweennessCentrality
0        349  Richardberg           11579.601583
1        128    Shanefort           10715.139355
2        700    Davisfort            5178.707487
3        815    Moodytown            4471.174851
4        555  Masseyhaven            3722.941825

Top 5 Airports for outDegreeCentrality
   airportId         name  outDegreeCentrality
0        815    Moodytown               2240.0
1        128    Shanefort               2195.0
2        700    Davisfort               2003.0
3        349  Richardberg               1104.0
4        485  Michaelstad                711.0

Top 5 Airports for inDegreeCentrality
   airportId         name  inDegreeCentrality
0        700    Davisfort              2205.0
1        128    Shanefort              1839.0
2        349  Richardberg              1312.0
3        815    Moodytown              1091.0
4        485  Michaelstad               758.0

Top 5 Airports for eigenve

In [10]:
#combined into a single table for top 5 betweenness centrality
gds.run_cypher(f'''
        MATCH(a:Airport)
        RETURN a.airportId AS airportId, a.name AS name,
            a.betweennessCentrality AS betweennessCentrality,
            a.outDegreeCentrality AS outDegreeCentrality,
            a.inDegreeCentrality AS inDegreeCentrality,
            a.eigenvectorCentrality AS eigenvectorCentrality
        ORDER BY betweennessCentrality DESC LIMIT {top_n}
''')

,airportId,name,betweennessCentrality,outDegreeCentrality,inDegreeCentrality,eigenvectorCentrality
0,349,Richardberg,11579.601583,1104.0,1312.0,0.455872
1,128,Shanefort,10715.139355,2195.0,1839.0,0.439031
2,700,Davisfort,5178.707487,2003.0,2205.0,0.643818
3,815,Moodytown,4471.174851,2240.0,1091.0,0.255747
4,555,Masseyhaven,3722.941825,275.0,204.0,0.046118


## Louvain Communities ("Regions") on UNDIRECTED Orientation

In [11]:
g, _ = gds.graph.project('proj', 'Airport', {'SENDS_TO':{'orientation':'UNDIRECTED', 'properties':['flightCount']}})
gds.louvain.write(g, relationshipWeightProperty='flightCount', writeProperty='louvainId')
g.drop()

Louvain:   0%|          | 0/100 [00:00<?, ?%/s]

graphName                                                         proj
database                                                         neo4j
memoryUsage                                                           
sizeInBytes                                                         -1
nodeCount                                                          237
relationshipCount                                                 2410
configuration        {'relationshipProjection': {'SENDS_TO': {'orie...
density                                                       0.043088
creationTime                       2023-01-21T17:11:20.105032000-05:00
modificationTime                   2023-01-21T17:11:20.160343000-05:00
schema               {'graphProperties': {}, 'relationships': {'SEN...
Name: 0, dtype: object

In [12]:
# Relabel Louvain Communities (a.k.a. Regions) by the highest betweenness centrality airport name
gds.run_cypher('''
    MATCH (a:Airport)
    WITH a.louvainId AS louvainId, max(a.betweennessCentrality) as maxBC
    MATCH(a:Airport) WHERE a.betweennessCentrality = maxBC
    WITH a.name as regionLabel, louvainId
    MATCH(a:Airport) WHERE a.louvainId=louvainId
    SET a.regionLabel = regionLabel
    RETURN count(a)
''')

,count(a)
0,237


In [13]:
# Count the number of airports in each region
gds.run_cypher('''
    MATCH (a:Airport)
    RETURN a.regionLabel as region, count(a) AS numberOfAirports
    ORDER BY region
''')

,region,numberOfAirports
0,Davisfort,32
1,Masseyhaven,50
2,Moodytown,51
3,Richardberg,48
4,Shanefort,56


## Using Graph Algorithms for Downstream Inference

In [14]:
# get the data
DELAY_CUTOFF = 30.0
df_raw = gds.run_cypher('''
    MATCH(a:Airport)<-[:LOCATED_AT]-()<-[r:TRANSPORT]-()-[:LOCATED_AT]->(b)
    WITH a, toInteger((r.effectiveMinutes-r.plannedMinutes) > $delayCutoff) AS wasDelayed,
        r.effectiveMinutes-r.plannedMinutes as delay,
        toInteger(a.regionLabel <> b.regionLabel) AS interRegion,
        b.outDegreeCentrality AS sourceOutDegree,
        b.name AS sourceName,
        b.airportId as sourceAirportId
    RETURN a.airportId as airportId,
        a.name AS name,
        sourceAirportId,
        sourceName,
        wasDelayed,
        delay,
        interRegion,
        sourceOutDegree,
        a.inDegreeCentrality AS inDegreeCentrality,
        a.outDegreeCentrality AS outDegreeCentrality,
        a.betweennessCentrality AS betweennessCentrality,
        a.eigenvectorCentrality AS eigenvectorCentrality,
        a.regionLabel AS regionLabel
''', params = {'delayCutoff': DELAY_CUTOFF})

In [15]:
df_raw.wasDelayed.value_counts()

0    13984
1     2183
Name: wasDelayed, dtype: int64

In [16]:
# subsample to airports with 100 or more arrivals
airports = df_raw.airportId.value_counts()
df = df_raw[df_raw.airportId.isin(airports[airports >= 100].index.to_list())]

In [17]:
# create airport fixed effects and intercept
df = pd.concat([df, pd.get_dummies(df.airportId, drop_first=True, prefix='airport')], axis=1)
df['intercept'] = 1

In [18]:
#log centrality variables
df['logBetweenness'] = np.log(df.betweennessCentrality + 1.0)
df['logEigenvector'] = np.log(df.eigenvectorCentrality + 1.0)
df['logInDegree'] = np.log(df.inDegreeCentrality + 1.0)
df['logOutDegree'] = np.log(df.outDegreeCentrality + 1.0)
df['logSourceOutDegree'] = np.log(df.sourceOutDegree + 1.0)

In [19]:
# check final class balance
df.wasDelayed.value_counts()

0    11084
1     1678
Name: wasDelayed, dtype: int64

In [20]:
df

,airportId,name,sourceAirportId,sourceName,wasDelayed,delay,interRegion,sourceOutDegree,inDegreeCentrality,outDegreeCentrality,...,airport_742,airport_770,airport_809,airport_815,intercept,logBetweenness,logEigenvector,logInDegree,logOutDegree,logSourceOutDegree
105,527,Bryanside,700,Davisfort,0,-249,1,2003.0,178.0,101.0,...,0,0,0,0,1,4.904083,0.030287,5.187386,4.624973,7.602900
106,527,Bryanside,700,Davisfort,0,-344,1,2003.0,178.0,101.0,...,0,0,0,0,1,4.904083,0.030287,5.187386,4.624973,7.602900
107,527,Bryanside,128,Shanefort,0,10,1,2195.0,178.0,101.0,...,0,0,0,0,1,4.904083,0.030287,5.187386,4.624973,7.694393
108,527,Bryanside,815,Moodytown,0,-117,0,2240.0,178.0,101.0,...,0,0,0,0,1,4.904083,0.030287,5.187386,4.624973,7.714677
109,527,Bryanside,815,Moodytown,0,-94,0,2240.0,178.0,101.0,...,0,0,0,0,1,4.904083,0.030287,5.187386,4.624973,7.714677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16099,485,Michaelstad,377,Rogersfurt,0,-101,0,45.0,758.0,711.0,...,0,0,0,0,1,7.482514,0.124103,6.632002,6.568078,3.828641
16100,485,Michaelstad,308,Fieldsbury,0,-100,1,274.0,758.0,711.0,...,0,0,0,0,1,7.482514,0.124103,6.632002,6.568078,5.616771
16101,485,Michaelstad,815,Moodytown,1,63,0,2240.0,758.0,711.0,...,0,0,0,0,1,7.482514,0.124103,6.632002,6.568078,7.714677
16102,485,Michaelstad,216,Davidburgh,0,-203,0,70.0,758.0,711.0,...,0,0,0,0,1,7.482514,0.124103,6.632002,6.568078,4.262680


In [21]:
# fit tme model
x_cols = ['logBetweenness',  'logInDegree', 'logOutDegree',
          'logEigenvector', 'interRegion','logSourceOutDegree', 'intercept']\
         + [i for i in df.columns if 'airport_' in i]
mod = sm.Logit(df.wasDelayed, df[x_cols])
res = mod.fit(cov_type='cluster', maxiter=10000, cov_kwds={'groups': df.airportId})
print(res.summary())

         Current function value: 0.339952
         Iterations: 10000


/Users/zachblumenfeld/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                           Logit Regression Results                           
Dep. Variable:             wasDelayed   No. Observations:                12762
Model:                          Logit   Df Residuals:                    12723
Method:                           MLE   Df Model:                           38
Date:                Sat, 21 Jan 2023   Pseudo R-squ.:                  0.1265
Time:                        17:12:37   Log-Likelihood:                -4338.5
converged:                      False   LL-Null:                       -4966.9
Covariance Type:              cluster   LLR p-value:                4.286e-239
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
logBetweenness         0.1269      0.039      3.231      0.001       0.050       0.204
logInDegree           -0.7110      0.039    -18.404      0.000      -0.787      -0.635
logOutDegree        

In [22]:
# Calculate predict probabilities and compare an example between Davisfort and Richardberg
df['prob'] = res.predict(df[x_cols])
df.loc[df.name.isin(["Davisfort", "Richardberg"]) & (df.sourceName == "Erictown"),["name", "prob"]].drop_duplicates()

,name,prob
5974,Davisfort,0.028762
12189,Richardberg,0.263881
